In [1]:
## written portion ##

Write down any 3 time series models that can be used for forecasting. (3 points)

ARIMA – Autoregressive Integrated Moving Average 
MA – Moving Average Models
AR - Autoregression Models

What are some real-world applications of time series forecasting? Name two. (2 points)

Businesses use sales data from previous sales over time to predict future sales

Weather data , previous temperatures, storm movements are used to forcast future weather.

What is the difference between a time series and a panel data set? (2 points)

Time series is one variable is one varbale observed over a certain time frame while panel data is multiple variables over a given time frame.

What is the major difference in how we select training sets in time series machine learning tasks compared to regression/classification tasks? (2 marks)

In time series training sets, the order matters. Chronological data is esential while in regression/classification tasks the time order shouldn't matter in the training set, the results shouldn't change because the data shouldn't vary over time.

Define the following terms as relates to time series models:

Stationarity (1 point)
When a time series of data has no long-term predictable patterns

Differencing (1 point)
stabilizing the variance of non stationary data. This operation calculates the change between error terms over consecutive time points.

what is White noise? (1 point)
When a series of data are distributed the same with a constant variance and mean of 0.

Random Walk (1 point)
An attempt to model time series data by creating random "steps" or jumps in the data over time as a way to predict changes over different intervals.

(5 points)
This is a generalized ARMA model The intercepts are as followed:
c is the constant
Epsilon Is the white noise
the Phi Is the coefficient for the auto-regression polynomial AR(1),AR(2),AR(3)
Theta is the coefficient for the moving average polynomial MA(1), MA(2), MA(3), MA(4)
Epsilon_t - 1 is the step or lags in the model for the MA portion of the ARMA.

Can you tell from the above equation whether the time series has been differenced?
Explain why or why not. (1 point)

From the Epsilon_t - 1 it looks like the time series has been differenced. I don't know how you could provide lags or an order if the time series were not differenced.

What is the Autoregressive (AR) order in the above equation? How about the Moving
Average (MA) order (2 points)

The AR order is AR(3),AR(2),AR(2), from the number above the summation (3) and t-1 denotion.
The MA order MA(4), MA(3), MA(2), MA(1) from the number above the summation (4) and t-1 denotion.

How do you determine the AR and MA order? How do you know how many lags to
include in your model? (2 points)

You determine the AR and MA order from the ACF and PACF plots of the differenced time series data. You look at the plots and chose the first significant bar, and any subsequent bar that's significant until a bar becomes insignificant.














In [2]:
import pandas as pd # v 1.5.3
import numpy as np # v 1.21.5
import statsmodels.api as sm # v 0.13.5
import pmdarima as pm #

import matplotlib.pyplot as plt # 3.6.3

import warnings as ww
import statsmodels.tsa.stattools as stattools  # v 0.13.5

from statsmodels.tsa.stattools import adfuller # v 0.13.5
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf # v 0.13.5
from statsmodels.tsa.arima_model import ARIMA # v 0.13.5

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error


In [3]:
df = pd.read_csv('/Users/samcampbell/Downloads/Electric_Production.csv')

In [4]:
### Below is some exploratory analysis ###

In [5]:
df.shape, df.columns, df.dtypes

## It looks like the date variable is an object, so I'll have to turn it into date type ##

((397, 2),
 Index(['DATE', 'IPG2211A2N'], dtype='object'),
 DATE           object
 IPG2211A2N    float64
 dtype: object)

In [6]:
df.head(),df.tail()

(       DATE  IPG2211A2N
 0  1/1/1985     72.5052
 1  2/1/1985     70.6720
 2  3/1/1985     62.4502
 3  4/1/1985     57.4714
 4  5/1/1985     55.3151,
           DATE  IPG2211A2N
 392   9/1/2017     98.6154
 393  10/1/2017     93.6137
 394  11/1/2017     97.3359
 395  12/1/2017    114.7212
 396   1/1/2018    129.4048)

In [7]:
df.isna().sum() 

DATE          0
IPG2211A2N    0
dtype: int64

In [8]:
df.describe()

,IPG2211A2N
count,397.000000
mean,88.847218
std,15.387834
min,55.315100
25%,77.105200
50%,89.779500
75%,100.524400
max,129.404800


In [9]:
df['DATE'] = pd.to_datetime(df['DATE'])

In [10]:
df.dtypes # changed date from an object to datetime

DATE          datetime64[ns]
IPG2211A2N           float64
dtype: object

In [11]:
df = df.sort_values('DATE') ## Sorting the values by date

In [12]:
df.head(),df.tail()

(        DATE  IPG2211A2N
 0 1985-01-01     72.5052
 1 1985-02-01     70.6720
 2 1985-03-01     62.4502
 3 1985-04-01     57.4714
 4 1985-05-01     55.3151,
           DATE  IPG2211A2N
 392 2017-09-01     98.6154
 393 2017-10-01     93.6137
 394 2017-11-01     97.3359
 395 2017-12-01    114.7212
 396 2018-01-01    129.4048)

In [13]:
## besides the data type for DATE not a lot stood out from my EDA, there aren't missing vals
## or anything that jumped out at me like that.

In [14]:
adf_test = sm.tsa.stattools.adfuller(df['IPG2211A2N'])

print('The p-value of the Dickey-Fuller test is:', adf_test[1])

print('The test statistic of the Dickey-Fuller test is:', adf_test[0])

kpss = stattools.kpss(df['IPG2211A2N'], regression='ct')

print('The p-value of the KPSS test is:', kpss[1])

print('The test statistic of the KPSS test is:', kpss[0])


### The p-values and test statistics from our Dickey Fuller and KPSS tests indicate that our -
### data is in fact non stationary so we need to difference it.


The p-value of the Dickey-Fuller test is: 0.18621469116586759
The test statistic of the Dickey-Fuller test is: -2.2569903500472455
The p-value of the KPSS test is: 0.01
The test statistic of the KPSS test is: 0.5051030832418743


/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:2019: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  warn_msg.format(direction="smaller"), InterpolationWarning


In [15]:
# undifferenced
df["IPG2211A2N"].plot(fig=plt.figure(i))
plt.title("IPG2211A2N")
plt.show() 

# The variance looks fairly high and the distribution looks skewed. This makes sense since-
# our data is not stationary and this is the undifferenced plot.

NameError: name 'i' is not defined

In [ ]:
#differenced once
df["IPG2211A2N"].diff().plot(fig=plt.figure(i))
plt.title("IPG2211A2N")
plt.show() 

# The variance and distribution look as if they will be tighter and more normal once-
# differenced, as expected.

In [ ]:
# differenced twice
df["IPG2211A2N"].diff().diff().plot(fig=plt.figure(i))
plt.title("IPG2211A2N")
plt.show() 

# Honestly this doesn't look much different than when I differenced the data once.

In [ ]:
## Here is a function for the ACF and PACF Plots ##
def acf_pacf (series)-> plt.plot:
    return plot_acf(series), plot_pacf(series)

In [ ]:
## I'm differencing my data for the plots here ##
acf_pacf(df['IPG2211A2N'].diff().tail(len(df)-1))
    


In [ ]:
## After running these plots and differencing the data it looks like the order for the ARMA/ARIMA
## woukld be (4,1,24)

In [ ]:
df['IPG2211A2N'] = df['IPG2211A2N'].diff() ## diferencing the data for the model ##

In [ ]:
model = sm.tsa.arima.ARIMA(df['IPG2211A2N'].head(int(0.7*len(df))), order=(4, 1, 24))
results = model.fit()

In [ ]:
results.summary()

In [ ]:
df['IPG2211A2N_forecast']=results.predict(start = int(0.7*len(df))+1, end=len(df) , dynamic=True)


In [ ]:
df[['IPG2211A2N','IPG2211A2N_forecast']].plot(figsize=(10,6))


In [ ]:
mean_absolute_error(df['IPG2211A2N_forecast'].tail(int(0.3*len(df))),df['IPG2211A2N'].tail(int(0.3*len(df))))


In [ ]:
mean_squared_error(df['IPG2211A2N_forecast'].tail(int(0.3*len(df))),df['IPG2211A2N'].tail(int(0.3*len(df))))


In [ ]:
## It's hard to interepet the mean errors for time series data like this relative to a universal-
## rule, but going off of the chart of the predicted and original data, the fit looks pretty good.
## some of the larger swings aren't accounted for so maybe differencing the data twice could help.